# <center>Web Scraping des annonces immobilières <center>

# <center>MUBAWAB.ma<center>

## <center>Région: Casablanca_Settat <center>

## Importation des bibliotèques

In [1]:
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver 
from selenium.webdriver.common.by import By
import re
import time

In [2]:
driver = webdriver.Chrome(r'C:\Users\taoul\OneDrive\Bureau\Stage 2A\Stage_terra\chromedriver.exe')

### Fonction d'extraction des urls des annonces d'une seul page

In [3]:
def get_page_links():
    links = driver.find_elements(By.XPATH,"//ul[@class='ulListing']/li")
    annonces_links = [i.get_attribute('linkref') for i in links]
    return(annonces_links)

In [4]:
## Extraction des urls des annonces du premièr URL ## 
driver.get('https://www.mubawab.ma/fr/plr/casablanca-settat/listing-promotion')
time.sleep(3)
annonces_links = get_page_links()

### Boucle while pour extraire les urls des annonces de toutes les pages

In [5]:
end = driver.find_element(By.XPATH,"//div[@class='ajaxPagination col-12']//p[@class='fSize11 centered']//span[@id='totalElementsSpan']").text
temp = driver.find_element(By.XPATH,"//div[@class='ajaxPagination col-12']//p[@class='fSize11 centered']//span[@id='lastElementSpan']").text
while(temp != end):
    try:
        end = driver.find_element(By.XPATH,"//div[@class='ajaxPagination col-12']//p[@class='fSize11 centered']//span[@id='totalElementsSpan']").text
        temp = driver.find_element(By.XPATH,"//div[@class='ajaxPagination col-12']//p[@class='fSize11 centered']//span[@id='lastElementSpan']").text
        actions = ActionChains(driver)
        elmt = driver.find_element(By.XPATH,"//div[@class='paginationDots sMargTop centered']//a[@id='nextPage']")
        driver.execute_script("arguments[0].click();", elmt)
        time.sleep(3)
        annonces_links.extend(get_page_links())
    except:
        pass

In [6]:
print("Nombre Total des Annonces: " + str(len(annonces_links)))

Nombre Total des Annonces: 424


## Données Extraites: 
- **Titre**:   Titre de l'annonce
- **Localisation**:   Localisation du bien immobilier 
- **Prix**:   Prix du bien (Soit 'Prix à consulter, Soit le Prix en chiffre du bien)
- **Standing**:   Standing du bien ('haut standing','moyen standing','economique')
- **Etat**:   Etat du bien ('finalisé', 'en cours de construction')
- **Date_Liv**:   Date de livraison
- **Latitude**:   Latitude de la position géographique du bien
- **Longitude**:   Longitude de la position géographique du bien
- **Link_Annonce**:   Lien de l'annonce
- **Nom_Annonceur**:   Nom de l'annonceur
- **Adresse_Annonceur**:   Adresse de l'annonceur
- **Tel_Annonceur**:   Numero de telephone de l'annonceur
- **Link_Annonceur**:   Lien de l'espace de l'annonceur sur MUBAWAB.ma


## Fonction d'extraction des données d'une annonce

In [18]:
def get_annonce_data(url):
    driver.get(url)
    url = url
    ################ Extraction du Titre de l'annonce ###################
    Titre = driver.find_element(By.XPATH,"//div[@class='agencyBox']//div/h1[@class='SpremiumH2']").text
    
    ################ Extraction de la localisation de l'annonce ###################
    Localisation = driver.find_element(By.XPATH,"//div[@class='agencyBox']//div//span[@class='immoDetails vAlignM darkblue']").text

    ################ Extraction du prix de l'annonce ###################
    try:
        Prix = driver.find_element(By.XPATH,"//div[@class='agencyBox']//div[@class = 'promotionInfoBox col-3']/h2").text
        Prix = int(re.findall(r'\d+', re.sub(' ','',Prix))[0])
    except:
        pass
    
    ################ Extraction du prix par m^2 ###################    
    try:
        Prix_U=driver.find_element(By.XPATH,"//div[@class='agencyBox']//div[@class = 'promotionInfoBox col-3']/span[@class='SpremiumDetails']").text
        Prix_U=int(re.findall(r'\d+', re.sub(' ','',Prix_U))[0])
    except:pass
    
    ################ Extraction de Standing, Etat et la Date de livraison de l'annonce ###################
    Infos = driver.find_elements(By.XPATH,"//div[@class='agencyBox']//div[@class = 'promotionInfoBox lHeight1 col-1 centered']//p")
    Infos = [i.text for i in Infos]
    Standings = ['haut standing','moyen standing','economique']
    Etats = ['finalisé', 'en cours de construction']
    Standing = ''
    Etat = ''
    for i in Infos:
        if i.lower() in Standings:
            Standing = i
        elif i.lower() in Etats:
            Etat = i
        elif 'Livraison' in i:
            Date_liv = i.replace('Livraison ','')
    Description_Annonce = driver.find_element(By.XPATH,"//div[@class='descrBlockProp descHolder']//p[@class='changeDescrip']").text

    ################ Extraction de la position géographiquel Latitde et Longitude ###################
    element = driver.find_element_by_link_text('Voir la carte')
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    driver.execute_script("arguments[0].click();", element)
    s=driver.find_element_by_id('mapHiddenForm').get_attribute("action")
    Latitude =float(s[s.find('&lat')+5:s.rfind('&lon')])
    Longitude  = float(s[s.find('&lon')+5:s.rfind('&zoom')])


    ####################### Informations de l'annonceur #####################################
    Nom_Annonceur = driver.find_element(By.XPATH,"//div[@class='agencyBox gradientLight']//div[@class='col-7']//div//h2").text
    Adresse_Annonceur = driver.find_element(By.XPATH,"//div[@class='agencyBox gradientLight']//div[@class='col-7']//div//p[@class='agencyAddress']").text
    Description_Annonceur = driver.find_element(By.XPATH,"//div[@class='agencyBox gradientLight']//div[@class='col-7']//div//p[@class='agencyText']").text
    Telephone_Annonceur = driver.find_elements(By.XPATH,"//div[@class='agencyBox gradientLight']//div[@class='col-7']//div[@class='contactLegend']//a")[0].get_attribute('href').replace('tel:','')
    Link_Annonceur = driver.find_elements(By.XPATH,"//div[@class='agencyBox gradientLight']//div[@class='col-7']//div[@class='contactLegend']//a")[1].get_attribute('href')
    
    ##################################### Return dict ###########################################
    return {'Titre':Titre, 'Localisation':Localisation,'Prix':Prix, 'Standing':Standing, 'Etat':Etat, 'Description_Annonce':Description_Annonce, 'Latitude':Latitude, 'Longitude':Longitude,'Link_Annonce':url, 'Nom_Annonceur':Nom_Annonceur, 'Adresse_Annonceur':Adresse_Annonceur, 'Description_Annonceur':Description_Annonceur, 'Telephone_Annonceur':Telephone_Annonceur, 'Link_Annonceur':Link_Annonceur}
    


## Boucle for pour l'extraction des données de l'ensemble des annonces

In [19]:
data=[]
for i in range(len(annonces_links)):
    try:
        data.append(get_annonce_data(annonces_links[i]))
    except:
        print(annonces_links[i])

## Création de la DataFrame ensemblant les données extraites

In [21]:
df= pd.DataFrame(data)
df.head(5)

,Titre,Localisation,Prix,Standing,Etat,Description_Annonce,Latitude,Longitude,Link_Annonce,Nom_Annonceur,Adresse_Annonceur,Description_Annonceur,Telephone_Annonceur,Link_Annonceur
0,La Vallée Dar Bouazza,Dar Bouazza,2400000,Haut standing,En cours de construction,"La Vallée, un complexe résidentiel de villas p...",33.510792,-7.823896,https://www.mubawab.ma/fr/p/3498/la-vall%C3%A9...,Al Akaria Développement,"BUREAU DE VENTE Adresse : KM 20, Route d'Azemm...",AL AKARIA Développement est une société de pro...,06 61 10 37 33,https://www.mubawab.ma/fr/b/4048/al-akaria-d%C...
1,Vert Marine 3,Dar Bouazza,1600000,Haut standing,En cours de construction,"Projet avant-gardiste de Saham Immobilier, Ver...",33.525617,-7.779887,https://www.mubawab.ma/fr/p/3512/vert-marine-3,Saham Immobilier - Vert Marine,Dar Bouazza,Saham Immobilier est la filiale de Saham Manag...,05 20 88 44 22,https://www.mubawab.ma/fr/b/7298/saham-immobil...
2,Les Perles Ain Sebaâ,Aïn Sebaâ à Casablanca,880000,Moyen standing,Finalisé,LA RÉSIDENCE LES PERLES AIN SEBAÂ offre un cad...,33.613905,-7.537994,https://www.mubawab.ma/fr/p/2772/les-perles-ai...,Projet immobilier,,,,https://www.mubawab.ma/fr/b/7294/projet-immobi...
3,Rosa Parc,Centre Ville à Mohammedia,661500,Haut standing,En cours de construction,Vivre dans une petite ville où règne sérénité ...,33.704193,-7.394542,https://www.mubawab.ma/fr/p/2918/rosa-parc,Groupe Allali - Rosa Parc,Mohammedia,"Fondé en 1989, le groupe Allali est aujourd’hu...",06 61 90 19 19,https://www.mubawab.ma/fr/b/7571/groupe-allali...
4,"Les Pavillons Anfa Club, by Yasmine Signature",Casablanca Finance City à Casablanca,1690000,Haut standing,Finalisé,Situé au sein du projet urbain Casa Anfa sur l...,33.547118,-7.675495,https://www.mubawab.ma/fr/p/3336/les-pavillons...,Yasmine Signature,Casa Finance City,Les Pavillons Anfa Club,06 60 88 63 56,https://www.mubawab.ma/fr/b/2804/yasmine-signa...


## Extraction des données sous format csv

In [23]:
df.to_csv('MUBAWAB_Data.csv')